In [16]:
import pandas as pd
import os
import cv2
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import ConfusionMatrixDisplay
import joblib
Categories = ["left", "right", "start", "stop"]
data_arr = []  # input array
target_arr = []  # output array
datadir = "./data/"


In [10]:
# path which contains all the categories of images
for i in Categories:
    print(f"loading... category : {i}")
    path = os.path.join(datadir, i)
    for img in os.listdir(path):
        img_array = imread(os.path.join(path, img))
        sample_image = cv2.resize(img_array, (128*4, 64*4))
        sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2GRAY)
        fd, hog_image = hog(
            sample_image,
            orientations=8,
            pixels_per_cell=(16, 16),
            cells_per_block=(1, 1),
            visualize=True,
        )
        data_arr.append(fd)
        target_arr.append(Categories.index(i))
    print(f"loaded category:{i} successfully")
data = np.array(data_arr)
target = np.array(target_arr)
df = pd.DataFrame(data)  # dataframe
df["Target"] = target
x = df.iloc[:, :-1]  # input data
y = df.iloc[:, -1]  # output data

loading... category : left
loaded category:left successfully
loading... category : right
loaded category:right successfully
loading... category : start
loaded category:start successfully
loading... category : stop
loaded category:stop successfully


In [17]:
# param_grid = {
#     "C": [0.1, 1, 10, 100],
#     "gamma": [0.0001, 0.001, 0.1, 1],
#     "kernel": ["rbf"],
# }
# svc = svm.SVC(probability=True)
# model = GridSearchCV(svc, param_grid)
model = SVC(C=10,gamma=0.1, kernel="rbf", probability=True)

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)
print('Splitted Successfully')
model.fit(x_train,y_train)
print('The Model is trained well with the given images')
# model.best_params_ contains the best parameters obtained from GridSearchCV

Splitted Successfully
The Model is trained well with the given images


In [19]:
filename = "finalized_model.sav"
joblib.dump(model, filename)
model = joblib.load("finalized_model.sav")

In [24]:
from sklearn.metrics import accuracy_score
y_pred=model.predict(x_test)
print("The predicted Data is :")
print(y_pred)
print("The actual data is:")
print(np.array(y_test))
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")

The predicted Data is :
[1 0 1 2 3 1 1 3 1 1 0 1 0 0 1 1 2 0 1 1 1 1 3 2 1 1 1 1 1 1 1 0 0 0 3 1 1
 2 1 1 1 0 1 1 3 1 0 3 1 1 2 1 1 1 3 1 3 1 1 1 1 1 2 1 1 1 0 1 1 1 1 0 0 1
 0 1 1 3 1 1 1 1 1 1 1 1 0 1 1 1 1 2 0 1 1 3 3 3 1 3 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 3 3 1 0 1 1 1 3 0 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 3
 0 0 1 1 1 3 1 1 0 0 1 1 3 1 1 2 1 1 1 1 0 3 1 3 1 0 3 1 1 1 1 2 0 0 1 1 1
 1 1 1 1 0 0 0 1 1 1 0 1 3 1 1 0 1 0 3 1 1 1 0 1 1 1 1 1 3 1 3 1 1 1 0 1 3
 1 1 1 1 1 1 2 2 0 1 3 3 0 3 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 3 1 0 1 0 1 0 1
 1 1 1 1 3 1 1 1 1 1 3 3 0 1 0 1 0 1 1 1 1 1 1 1 3 1 1 0 1 3 1 1 3 1 0 1 1
 0 1 0 0 1 0 3 0 1 1 1 1 1 0 2 0 1 2 1 1 1 0 1 1 0 1 1 1 1 1 2 0 1 1 3 1 0
 1 0 3 1 1 1 1 3 0 0 1 1 1 0 1 1 1 0 1 1 1 1 3 1 2 3 1 1 1 1 1 1]
The actual data is:
[1 0 2 2 3 1 2 3 2 3 0 1 0 0 1 1 2 0 2 2 1 1 3 2 2 1 1 1 1 1 1 0 0 0 3 2 2
 2 1 2 3 0 1 1 3 2 0 3 1 1 2 0 1 3 3 2 3 2 2 1 1 1 2 1 2 2 0 2 3 2 1 0 0 1
 0 1 2 3 1 0 1 3 3 1 2 2 0 1 2 2 1 2 0 1 1 3 3 3 1 3 3 0 1 1 2 0 

In [ ]:
titles_options = [
    ("Confusion matrix, without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        x_test,
        y_test,
        display_labels=Categories,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix) # type: ignore

plt.show()

In [ ]:
sample_image = cv2.imread("./PXL_20230524_175015234.jpg")
sample_image = cv2.resize(sample_image, (128*4, 64*4))
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2GRAY)
plt.imshow(sample_image, cmap=plt.cm.gray)

In [ ]:
fd, hog_image = hog(
    sample_image,
    orientations=8,
    pixels_per_cell=(16, 16),
    cells_per_block=(1, 1),
    visualize=True,
)
probability=model.predict_proba([fd])
for ind,val in enumerate(Categories):
    print(f'{val} = {probability[0][ind]*100}%')
print("The predicted image is : "+ Categories[model.predict([fd])[0]])
plt.imshow(hog_image)

In [ ]:
video = cv2.VideoCapture(0)

success = True


while success:
    success , frame = video.read()
    
    if success == True:
        
        
        # i want every 5th frame from video
        # thats why i used following line of code
        # i dont want all frames from video
        # so we can decide the outpt frames count according to us.
        
        
        sample_frame = cv2.resize(frame, (128*4, 64*4))
        sample_frame = cv2.cvtColor(sample_frame, cv2.COLOR_BGR2GRAY)
        fd, hog_image = hog(
        sample_frame,
        orientations=8,
        pixels_per_cell=(16, 16),
        cells_per_block=(1, 1),
        visualize=True,
    )
        probability=model.predict_proba([fd])
        for ind,val in enumerate(Categories):
            print(f'{val} = {probability[0][ind]*100}%')
        print("The predicted image is : "+ Categories[model.predict([fd])[0]])
        cv2.imshow("Image", sample_frame)
        if cv2.waitKey(1) == ord('q'):
            break
    else:
        break